# Transition System: TicTacToe

This tutorial describes how to define two-player turn-based transition systems using IGLSynth. We will use the example of TicTacToe. 


## TicTacToe Model

A transition system models the interaction of two-players as a directed (multi-)graph with action-labeled edges.

We will start by defining a mathematical model for transition system of TicTacToe. 
In general, we need two ingredients to define such a model, namely

* States
* Actions

A state of the TicTacToe transition system must represent two parameters: (a) the configuration of the board, and (b) whose turn it is to play. 


**Representing States**

The configuration of board can easily be captured in a 9-tuple such that the $i$-th index of tuple corresponds to a cell as shown below

                        0 | 1 | 2

                        3 | 4 | 5

                        6 | 7 | 8


Given the number of `X` and `O` marks on the board, it is easy to determine whose turn it is to play next. `X` plays when $|\mathsf{X}| = |\mathsf{O}|$ and `O` plays when $|\mathsf{X}| > |\mathsf{O}|$. 

Based on above two arguments, we can simply represent the state of TicTacToe transition system as a 9-tuple with `None` (to represent unmarked cell), `X` or `O`. 


**Representing Actions**

The actions in TicTacToe can be represented as $mark_i$ for $i = 0, \ldots, 8$. 
When an action $mark_i$ is performed in a state $s$, the $i$-th cell is marked `X` if it is P1's turn to play at $s$, and `O` otherwise. 


With this model in mind, we start to write an IGLSynth code to represent the `TSys`. 



## Instantiating TSys

In IGLSynth, a transition system is modeled using `model` package, which contains the `TSys` class. By default, `TSys` represents a deterministic two-player transition system, which can be defined to be turn-based by setting the initialization parameter `kind` to `TURN_BASED` (default is `CONCURRENT`).


Let us start by importing the `model` package.

In [1]:
import sys
sys.path.insert(0, '/home/abhibp1993/Research/iglsynth')

In [2]:
import iglsynth.model as model

A turn-based transition system can be instantiated by providing a `kind` and a `name`. 

If the `name` parameter is not provided, a name is automatically assigned (as `TSys` is an `Entity`). However, the automatically assigned names are tedious remember, hence we recommend setting the name to something meaningful, say `tictactoe`. 

<div class="alert alert-block alert-warning">
<b>Remark:</b> By default, all transition systems in IGLSynth are assumed to be two-player. 
</div>


In [3]:
tsys = model.TSys(kind=model.TURN_BASED, name="tictactoe")
tsys

TSys(name=tictactoe, |V|=0, |E|=0)

In general, `TSys` constructor can take several arguments to initialize the object. We refer the readers to documentation for their details. 


## Adding States to TSys

The states that are added to `tsys` must be of `tsys.Vertex` type. By default, `tsys.Vertex` class is an `Entity` that derives from `util.Vertex` class which takes in `state` and `turn` as constructor arguments. Internally, the `name` of vertex is determined as the tuple `(state, turn)`. 

Thus, we first define a method to determine the turn of player given a state and then generate the set of vertices.


**Note:** Here, we take a very naive combinatorial approach to generating states for illustrative purposes. Many of these states will not be reachable. 

In [4]:
def turn(tup):
    if tup.count("X") == tup.count("O"):
        return model.TURN_P1
    else:
        return model.TURN_P2

In [5]:
import itertools
from tqdm import tqdm

# Generate set of vertices
tokens = ("X", "O", None)
vertices = []
for tup in tqdm(tuple(itertools.product(tokens, repeat=9))):
    v = tsys.Vertex(state=tup, turn=turn(tup))
    vertices.append(v)

# Add the states to tsys
tsys.add_vertices(vertices)

# Print tsys
print(tsys)

100%|██████████| 19683/19683 [00:00<00:00, 44515.31it/s]


TSys(name=tictactoe, |V|=19683, |E|=0)


# Adding Edges to TSys

There are two prevalent approaches to construct edges of TSys. 
* Combinatorial
* Iterative

**In combinatorial approach**, we define a function `is_valid(u, v, a) --> bool` that checks whether the given edge between two vertices $u \in V$ and $v \in V$ is valid or not. A typical way to implement this would be as follows:

```python
V = tuple(tsys.vertices)
for u, v in itertools.product(V, V):
    for a in tsys.act:
        if is_valid(tsys.Edge(u, v, a)):
            tsys.add_edge(tsys.Edge(u, v, a))
```

**In iterative approach**, we construct edges iteratively applying actions (`tsys.act`) to a set of initial states to generate a set of new states. If any of the new states are not in `tsys`, we discard them. Otherwise, we apply actions to these states. We repeat this process until we do not generate any new states. A typical way to implement this would be as follows:

```python
stack = [v0, ..., vn]
explored = set()
while len(stack) > 0:
    u = stack.pop()
    explored.add(u)
    
    for a in tsys.act:
        v = a(u)
        
        if v in tsys and v not in explored:
            e = tsys.Edge(u, v, a)
            tsys.add_edge(e)
            stack.append(u)
```

As a thumb rule, the iterative approach works better when the size of action set is much smaller than the size of state space. The combinatorial approach is suitable when either the size action set is comparable to that of state space or we are using parallelization. 

Thus, in our `tictactoe` example, we will use the iterative approach. 

In [6]:
v0 = tsys.Vertex(state=(None, )*9, turn=model.TURN_P1)

stack = [v0]
explored = set()

while len(stack) > 0:
    u = stack.pop()
    explored.add(u)
    
    for a in tsys.act:
        v = a(u)
        explored.add(u)
        
        if v in tsys and v not in explored:
            e = tsys.Edge(u, v, a)
            tsys.add_edge(e)
            stack.append(u)

# Print transition system
tsys

TSys(name=tictactoe, |V|=19683, |E|=0)

Observe that **no** edges were added to the `tsys`. This is because we haven't added any actions to transition system. So let us do that. 


### Defining Actions

There are two types of action objects in IGLSynth, namely `Action` and `ConcurrentAction`. As suggested by the name `ConcurrentAction` represents an action where both simultaneously effect the state. On the other hand, `Action` class represents the turn-based actions. As `tictactoe` is turn-based, we will define a set of `Action` objects. 


An `Action` comprises of a precondition and a postcondition. Both the precondition and postcondition are functions with the following signatures:
* Precondition: `precondition(v, *args, **kwargs) -> bool`
* Postcondition: `postcondition(v, *args, **kwargs) -> 'util.Vertex'`

The purpose of the `precondition` function is to check whether the action is applicable at the given vertex or not. The `postcondition` function is expected to return the result of performing the action at the given vertex. 

Let us look at an example. 

In [7]:
@model.Action
def mark0(v):
    tup = v.name[0]     # Recall v.name has structure (state, turn)
    
    if v.turn == model.TURN_P1:
        new_tup = ("X", ) + tup[1:]
    else:
        new_tup = ("O", ) + tup[1:]
    
    return tsys.Vertex(state=new_tup, turn=turn(new_tup))

# A test case (let us apply on v0)
v = mark0(v0)
v

TSysVertex(name=(('X', None, None, None, None, None, None, None, None), 'TURN_P2'))

Let us note the following aspects of above code.

* We defined `mark0` using the decorator `@model.Action`. Behind the scenes, this instantiates an `Action` with postcondition defined by the `mark0` function. 

* We "called" the action `mark0` as a function with argument `v0` to compute the result of applying `mark0` at `v0`. Behind the scenes, `mark0(v0)` first evaluates whether `mark0.precondition(v0)` is `True` or `False` (default: `True`). Only when precondition evaluates to `True`, the postcondition is evaluated and returned. 


Let us see an example of how to add a precondition to `mark0`. Say, `mark0` should not be applied to a state if cell 0 is already marked. 

In [8]:
# Define the precondition

@mark0.precondition
def mark0(v):
    tup = v.name[0]     # Recall v.name has structure (state, turn)
    
    if tup[0] is None:
        return True
    
    return False


# Apply mark0 on output of last block
v = mark0(v0)
v = mark0(v)
print(v)

None


Observe that `mark0(v)` was returned as `None` representing that the precondition of the action did not evaluate to `True`. 


Let us define actions `mark1, ..., mark8` similar to `mark0`.

**Note:** It is possible to use `map` function in conjunction with `functools.partial` to efficiently define 9 actions in a few lines. But for thre sake of simpler exposition, we explicitly define all actions.

In [9]:
# Define generalized precondition and postcondition functions
def mark_precondition(v, i):
    tup = v.name[0]     # Recall v.name has structure (state, turn)
    
    if tup[i] is None:
        return True
    
    return False

def mark_postcondition(v, i):
    tup = v.name[0]
    
    if v.turn == model.TURN_P1:
        new_tup = list(tup)
        new_tup[i] = "X"
        new_tup = tuple(new_tup)
    else:
        new_tup = list(tup)
        new_tup[i] = "O"
        new_tup = tuple(new_tup)
    
    return tsys.Vertex(state=new_tup, turn=turn(new_tup))


# Define actions (postconditions)
@model.Action
def mark0(v):
    return mark_postcondition(v, 0)

@model.Action
def mark1(v):
    return mark_postcondition(v, 1)
    
@model.Action
def mark2(v):
    return mark_postcondition(v, 2)

@model.Action
def mark3(v):
    return mark_postcondition(v, 3)
    
@model.Action
def mark4(v):
    return mark_postcondition(v, 4)
    
@model.Action
def mark5(v):
    return mark_postcondition(v, 5)
    
@model.Action
def mark6(v):
    return mark_postcondition(v, 6)
    
@model.Action
def mark7(v):
    return mark_postcondition(v, 7)
    
@model.Action
def mark8(v):
    return mark_postcondition(v, 8)
    

# Define preconditions
@mark0.precondition
def mark0(v):
    return mark_precondition(v, 0)

@mark1.precondition
def mark1(v):
    return mark_precondition(v, 1)

@mark2.precondition
def mark2(v):
    return mark_precondition(v, 2)
    
@mark3.precondition
def mark3(v):
    return mark_precondition(v, 3)

@mark4.precondition
def mark4(v):
    return mark_precondition(v, 4)

@mark5.precondition
def mark5(v):
    return mark_precondition(v, 5)
    
@mark6.precondition
def mark6(v):
    return mark_precondition(v, 6)

@mark7.precondition
def mark7(v):
    return mark_precondition(v, 7)

@mark8.precondition
def mark8(v):
    return mark_precondition(v, 8)
    

# Define the set of all actions
actions = [mark0, mark1, mark2, mark3, mark4, mark5, mark6, mark7, mark8]
tsys.act = actions

Now that we have defined all actions, let us use these actions to generate the edges of `tsys`. 

In [10]:
v0 = tsys.Vertex(state=(None, )*9, turn=model.TURN_P1)

stack = [v0]
explored = set()

while len(stack) > 0:
    u = stack.pop()
    explored.add(u)
    
    for a in tsys.act:
        v = a(u)
        if v is None:       # <----- NEW! Actions return None when preconditions fail. 
            continue 
            
        explored.add(u)
        if v in tsys:
            e = tsys.Edge(u, v, a)
            tsys.add_edge(e)
        
        if v not in explored:
            stack.append(v)

# Print transition system
tsys

TSys(name=tictactoe, |V|=19683, |E|=19107)

Once we have identified reachable states from the initial state `v0`, it's often convenient to prune out the remaining vertices to obtain a compact graph for transition system. 

In our case, pruning is straightforward --- remove any state with no incoming or outgoing edge. 

In [11]:
# Prune
rem_vertices = []
for v in tsys.vertices:
    if len(tuple(tsys.out_edges(v))) == 0 and len(tuple(tsys.in_edges(v))) == 0:
        rem_vertices.append(v)

tsys.rm_vertices(rem_vertices)
tsys

TSys(name=tictactoe, |V|=6046, |E|=19107)

In [12]:
# Let us check whether our tsys is correct. 

# Case 1: Out edges at initial state
v0 = tsys.Vertex(state=(None, )*9, turn=turn((None, )*9))
assert len(tuple(tsys.out_edges(v0))) == 9
assert len(tuple(tsys.in_edges(v0))) == 0

# Case 2: Out edges at some intermediate state
state = ('X', None, 'O', None, None, None, 'X', None, None)
v0 = tsys.Vertex(state=state, turn=turn(state))
print()
for e in tuple(tsys.out_edges(v0)):
    print(e)
print()
for e in tuple(tsys.in_edges(v0)):
    print(e)


TSysEdge(name=(TSysVertex(name=(('X', None, 'O', None, None, None, 'X', None, None), 'TURN_P2')), TSysVertex(name=(('X', None, 'O', None, 'O', None, 'X', None, None), 'TURN_P1')), Action(name=mark4)))
TSysEdge(name=(TSysVertex(name=(('X', None, 'O', None, None, None, 'X', None, None), 'TURN_P2')), TSysVertex(name=(('X', None, 'O', None, None, None, 'X', 'O', None), 'TURN_P1')), Action(name=mark7)))
TSysEdge(name=(TSysVertex(name=(('X', None, 'O', None, None, None, 'X', None, None), 'TURN_P2')), TSysVertex(name=(('X', None, 'O', None, None, 'O', 'X', None, None), 'TURN_P1')), Action(name=mark5)))
TSysEdge(name=(TSysVertex(name=(('X', None, 'O', None, None, None, 'X', None, None), 'TURN_P2')), TSysVertex(name=(('X', None, 'O', None, None, None, 'X', None, 'O'), 'TURN_P1')), Action(name=mark8)))
TSysEdge(name=(TSysVertex(name=(('X', None, 'O', None, None, None, 'X', None, None), 'TURN_P2')), TSysVertex(name=(('X', None, 'O', 'O', None, None, 'X', None, None), 'TURN_P1')), Action(name=mar

Voila! We have our transition system for tictactoe. 


**Reference:** It is possible to obtain even more compact transition system based on the [Math StackExchange discussion](https://math.stackexchange.com/questions/485752/tictactoe-state-space-choose-calculation/485852). 